## Graph Convolutional Network
#### List of GNN Applications:

* Node classification: The objective here is to predict the labels of nodes by considering the labels of their neighbors. 
* Link prediction: In this case, the goal is to predict the relationship between various entities in a graph. This can for example be applied in prediction connections for social networks. 
* Graph clustering: This involves dividing the nodes of a graph into clusters. The partitioning can be done based on edge weights or edge distances or by considering the graphs as objects and grouping similar objects together. 
* Graph classification: This entails classifying a graph into a category. This can be applied in social network analysis and categorizing documents in natural language processing. Other applications in NLP include text classification, extracting semantic relationships between texts, and sequence labeling. 
* Computer vision: In the computer vision world, GNNs can be used to generate regions of interest for object detection. They can also be used in image classification whereby a scene graph is generated. The scene generation model then identifies objects in the image and the semantic relationship between them. Other applications in this field include interaction detection and region classification. 

#### GCN implementation with DGL

One of the most popular and widely adopted tasks on graph data is node classification, where a model needs to predict the ground truth category of each node. Before graph neural networks, many proposed methods are using either connectivity alone (such as DeepWalk or node2vec), or simple combinations of connectivity and the node’s own features. GNNs, by contrast, offers an opportunity to obtain node representations by combining the connectivity and features of a local neighborhood.

# Node classification

In [1]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.8 MB/s 
     |████████████████████████████████| 291 kB 52.0 MB/s 
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [2]:
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.function as fn
from dgl import DGLGraph

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


The forward function is essentially the same as any other commonly seen NNs model in PyTorch. We can initialize GCN like any nn.Module. For example, let’s define a simple neural network consisting of two GCN layers. Suppose we are training the classifier for the cora dataset (the input feature size is 1433 and the number of classes is 7). The last GCN layer computes node embeddings, so the last layer in general does not apply activation.

In [3]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

We load the cora dataset using DGL’s built-in data module.
The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.

In [4]:
from dgl.data import CoraGraphDataset

def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    return g, features, labels, train_mask, test_mask, dataset.num_classes

In [5]:
g, features, labels, train_mask, test_mask, num_classes = load_cora_data()
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


When a model is trained, we can use the following method to evaluate the performance of the model on the test dataset:

In [6]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

Task: Complete the training process and run and check the accuracy.

In [10]:
# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, num_classes)
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
for epoch in range(50):

    model.train()

    # Write 3 lines of code to call the network on the graph
    logits = model(g, features)
    # Apply log_softmax on the logits
    logits = F.log_softmax(logits)
    # Apply NLL_loss on the scores while applying train_mask
    loss = F.nll_loss(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = evaluate(model, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f}".format(
            epoch, loss.item(), acc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  
/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00000 | Loss 1.9462 | Test Acc 0.1560
Epoch 00001 | Loss 1.9380 | Test Acc 0.3830
Epoch 00002 | Loss 1.9288 | Test Acc 0.6010
Epoch 00003 | Loss 1.9169 | Test Acc 0.6300
Epoch 00004 | Loss 1.9031 | Test Acc 0.5410
Epoch 00005 | Loss 1.8881 | Test Acc 0.4740
Epoch 00006 | Loss 1.8728 | Test Acc 0.4710
Epoch 00007 | Loss 1.8566 | Test Acc 0.4850
Epoch 00008 | Loss 1.8387 | Test Acc 0.4940
Epoch 00009 | Loss 1.8194 | Test Acc 0.4930
Epoch 00010 | Loss 1.7998 | Test Acc 0.5000
Epoch 00011 | Loss 1.7790 | Test Acc 0.5210
Epoch 00012 | Loss 1.7568 | Test Acc 0.5440
Epoch 00013 | Loss 1.7328 | Test Acc 0.5540
Epoch 00014 | Loss 1.7075 | Test Acc 0.5590
Epoch 00015 | Loss 1.6817 | Test Acc 0.5690
Epoch 00016 | Loss 1.6552 | Test Acc 0.5890
Epoch 00017 | Loss 1.6277 | Test Acc 0.6030
Epoch 00018 | Loss 1.5991 | Test Acc 0.6150
Epoch 00019 | Loss 1.5696 | Test Acc 0.6250
Epoch 00020 | Loss 1.5392 | Test Acc 0.6350
Epoch 00021 | Loss 1.5081 | Test Acc 0.6490
Epoch 00022 | Loss 1.4761 | Test

# Link prediction
Many applications such as social recommendation, item recommendation, knowledge graph completion, etc., can be formulated as link prediction, which predicts whether an edge exists between two particular nodes. This tutorial shows an example of predicting whether a citation relationship, either citing or being cited, between two papers exists in a citation network.

In [8]:
import itertools
import scipy.sparse as sp

### Prepare training and testing sets

In [11]:
g, features, labels, train_mask, test_mask, num_classes = load_cora_data()

# Split edge set for training and testing
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [12]:
train_g = dgl.remove_edges(g, eids[:test_size])

We build a model consisting of two GraphSAGE layers, each computes new node representations by averaging neighbor information. DGL provides dgl.nn.SAGEConv that conveniently creates a GraphSAGE layer.

In [13]:
from dgl.nn import SAGEConv

# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

The model then predicts the probability of existence of an edge by computing a score between the representations of both incident nodes with a function (e.g. an MLP or a dot product).

DGL recommends you to treat the pairs of nodes as another graph, since you can describe a pair of nodes with an edge. In link prediction, you will have a positive graph consisting of all the positive examples as edges, and a negative graph consisting of all the negative examples. The positive graph and the negative graph will contain the same set of nodes as the original graph. This makes it easier to pass node features among multiple graphs for computation. As you will see later, you can directly feed the node representations computed on the entire graph to the positive and the negative graphs for computing pair-wise scores.

The following code constructs the positive graph and the negative graph for the training set and the test set respectively.

In [14]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

The benefit of treating the pairs of nodes as a graph is that you can use the DGLGraph.apply_edges method, which conveniently computes new edge features based on the incident nodes’ features and the original edge features (if applicable).

DGL provides a set of optimized builtin functions to compute new edge features based on the original node/edge features. For example, dgl.function.u_dot_v computes a dot product of the incident nodes’ representations for each edge.

In [15]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

You can also write your own function if it is complex. For instance, the following module produces a scalar score on each edge by concatenating the incident nodes’ features and passing it to an MLP.

In [16]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [17]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)

# You can replace DotPredictor with MLPPredictor.
# pred = MLPPredictor(16)

pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    # define labels, 1 line
    labels = torch.cat((torch.ones(pos_score.shape), torch.zeros(neg_score.shape)))
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    # define labels, 1 line
    labels = torch.cat((torch.ones(pos_score.shape), torch.zeros(neg_score.shape)))
    return roc_auc_score(labels, scores)

In [18]:
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])

    # predict train pos and train neg scores, and compute loss, 3 lines
    train_pos_pred = pred(train_pos_g, h)
    train_neg_pred = pred(train_neg_g, h)
    loss = compute_loss(train_pos_pred, train_neg_pred)
    
    # loss = compute_loss()

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))


from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.6929841041564941
In epoch 5, loss: 0.6652697920799255
In epoch 10, loss: 0.576982855796814
In epoch 15, loss: 0.5071618556976318
In epoch 20, loss: 0.4702264666557312
In epoch 25, loss: 0.4362088739871979
In epoch 30, loss: 0.41358429193496704
In epoch 35, loss: 0.38888028264045715
In epoch 40, loss: 0.3662951588630676
In epoch 45, loss: 0.3438146114349365
In epoch 50, loss: 0.3205651044845581
In epoch 55, loss: 0.29764366149902344
In epoch 60, loss: 0.274132639169693
In epoch 65, loss: 0.25071215629577637
In epoch 70, loss: 0.2272685319185257
In epoch 75, loss: 0.2039857655763626
In epoch 80, loss: 0.18112502992153168
In epoch 85, loss: 0.15937282145023346
In epoch 90, loss: 0.13829749822616577
In epoch 95, loss: 0.11842048913240433
AUC 0.8489899148716336
